In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers
!pip install --upgrade datasets

In [ ]:
import pandas as pd
from datasets import Dataset

# Load dataset
file_path = '/content/drive/MyDrive/ML_project/merged_data.csv'
df = pd.read_csv(file_path)


incorrect_df = pd.DataFrame({'text': df['sentence'], 'label': 0})  # Incorrect labeled as 0
correct_df = pd.DataFrame({'text': df['corrections'], 'label': 1})  # Correct labeled as 1

# ✅ Combine both into one dataset
final_df = pd.concat([incorrect_df, correct_df], ignore_index=True)

# ✅ Shuffle the dataset
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Verify the mapping
print(final_df.head())
print(final_df['label'].value_counts())

                                                text  label
0  Even if the status of cars are still at the pr...      1
1  25. There's no such thing as a free lunch, but...      0
2                          It's a hard nut to crack.      1
3  Also , food chain becomes more stronger that i...      0
4  Kindly refrain from makin' noise during the pr...      0
label
1    2772
0    2772
Name: count, dtype: int64


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset

# ✅ Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Adjust `num_labels` to match your dataset

# ✅ Convert pandas DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(final_df)

# ✅ Define the Tokenization Function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],         # Tokenize the 'text' column
        truncation=True,          # Truncate sequences longer than the model's max input length
        padding='max_length',     # Pad sequences to the model's max input length
        max_length=64             # Set maximum token length
    )

# ✅ Apply Tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# ✅ Ensure Correct Dataset Format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# ✅ Verify Tokenization
print(tokenized_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5544 [00:00<?, ? examples/s]

{'label': tensor(1), 'input_ids': tensor([  101,  2130,  2065,  1996,  3570,  1997,  3765,  2024,  2145,  2012,
         1996, 12825,  7021,  2004, 14874,  1010,  2242,  2842,  2097,  1010,
         1997,  2607,  1010,  2202,  1996,  2173,  1997,  3765,  1999,  2256,
         3268,  1010,  1998,  2381,  4136,  2149,  2008, 12607,  2097,  2196,
         2644,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


In [ ]:
# Check label distribution
unique_labels = final_df['label'].unique()
print("Unique Labels:", unique_labels)
print("Number of Unique Labels:", len(unique_labels))
print("Label Min:", final_df['label'].min(), "Label Max:", final_df['label'].max())


Unique Labels: [1 0]
Number of Unique Labels: 2
Label Min: 0 Label Max: 1


In [ ]:
print(final_df['label'].unique())
print(final_df['label'].min(), final_df['label'].max())

[1 0]
0 1


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Define a function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")  # Use weighted average for multi-class
    return {"f1": f1}

In [ ]:

from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

# ✅ Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ✅ Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Add this line

)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.590600,0.479779,0.794669
2,0.441500,0.355664,0.862329
3,0.334600,0.266182,0.898953
4,0.451000,0.194482,0.926830
5,0.181200,0.181907,0.935382


TrainOutput(global_step=3465, training_loss=0.40743142134183413, metrics={'train_runtime': 729.064, 'train_samples_per_second': 38.021, 'train_steps_per_second': 4.753, 'total_flos': 911679806822400.0, 'train_loss': 0.40743142134183413, 'epoch': 5.0})

In [ ]:
print(model.config.architectures)

['BertForSequenceClassification']


In [ ]:
import torch

def classify_sentence(sentence):
    model.eval()
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        max_length=64,
        truncation=True,
        padding='max_length'
    ).to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # Shape: [1, num_labels]
        preds = torch.argmax(logits, dim=-1)  # Get the predicted class index

    return "Correct" if preds.item() == 1 else "Incorrect"

# Test Example
test_sentence = "I is happy to see you."
classification = classify_sentence(test_sentence)
print("Sentence:", test_sentence)
print("Classification:", classification)


Sentence: I is happy to see you.
Classification: Incorrect


In [ ]:
# Expanded Quick Accuracy Check Sentences
test_cases = [
    "She will goes to the market yesterday but forgot her wallet.", "She went to the market yesterday, but forgot her wallet.",
    "Me and him goes to store and buyed candies.", "He and I went to the store and bought candies.",
    "He don't knows nothing about fixing cars.", "He doesn't know anything about fixing cars.",
    "Although he was tired. Decided to keep working.", "Although he was tired, he decided to keep working.",
    "Their going too the park later, isn't it?", "They're going to the park later, aren't they?",
    "The dog bark loudly every morning.", "The dog barks loudly every morning.",
    "He didn't knew about the meeting.", "He didn't know about the meeting.",
    "Me and her went to the cinema.", "She and I went to the cinema.",
    "She don't wants to go out tonight.", "She doesn't want to go out tonight.",
    "We was watching a movie yesterday.", "We were watching a movie yesterday.",
    "I didn't see nobody at the party.", "I didn't see anybody at the party.",
    "She is good in math.", "She is good at math.",
    "The flowers smells nice.", "The flowers smell nice.",
    "Running fast, the finish line was reached.", "Running fast, he reached the finish line.",
    "She buyed a new dress last week.", "She bought a new dress last week.",
    "Them is going to the event.", "They are going to the event.",
    "He was so tired, he slept all the day.", "He was so tired that he slept all day.",
    "I seen him at the mall yesterday.", "I saw him at the mall yesterday.",
    "She enjoy to read books.", "She enjoys reading books.",
    "Her don't like spicy food.", "She doesn't like spicy food.",
    "They was playing soccer when I arrived.", "They were playing soccer when I arrived.",
    "Him is taller than me.", "He is taller than me.",
    "The book were on the table.", "The book was on the table.",
    "He don't have no money.", "He doesn't have any money.",
    "The cat was sitting on a tree who was very high.", "The cat was sitting on a tree that was very high.",
    "She is married with a lawyer.", "She is married to a lawyer.",
    "He doesn't knows where the library is.", "He doesn't know where the library is.",
    "While walking through the park, the birds sang beautifully.", "While walking through the park, he heard the birds singing beautifully.",
    "Me and my friend likes pizza.", "My friend and I like pizza.",
    "He was interested on the new project.", "He was interested in the new project.",
    "I cannot able to do it.", "I cannot do it.",
    "She is knowing the answer.", "She knows the answer.",
    "If I was you, I would apologize.", "If I were you, I would apologize.",
    "The movie was too much long.", "The movie was too long.",
    "I am agree with your opinion.", "I agree with your opinion.",
    "She said me to come early.", "She told me to come early.",
    "The train leave at 5 PM every day.", "The train leaves at 5 PM every day.",
    "I have seen her yesterday.", "I saw her yesterday.",
    "He suggested me to join the team.", "He suggested that I join the team.",
    "They was not ready for the trip.", "They were not ready for the trip.",
    "Neither of the boys are responsible.", "Neither of the boys is responsible.",
    "She sings good.", "She sings well.",
    "He is very interesting in science.", "He is very interested in science.",
    "Me went to the party alone.", "I went to the party alone.",
    "He didn't knew what to say.", "He didn't know what to say.",
    "The team are playing well.", "The team is playing well.",
    "You should to try harder.", "You should try harder.",
    "He is more taller than me.", "He is taller than me.",
    "She do her homework every evening.", "She does her homework every evening.",
    "Each of the students were given a book.", "Each of the students was given a book.",
    "He doesn't likes coffee.", "He doesn't like coffee.",
    "This place is more better than the other one.", "This place is better than the other one.",
    "I wish I was there.", "I wish I were there.",
    "He has went to the market.", "He has gone to the market.",
    "The dog is barking on the stranger.", "The dog is barking at the stranger.",
    "He explained me the problem.", "He explained the problem to me.",
    "She hasn't no time to talk.", "She has no time to talk.",
    "I am looking forward to meet you.", "I am looking forward to meeting you.",
    "She can sings beautifully.", "She can sing beautifully.",
    "He don't likes working on weekends.", "He doesn't like working on weekends.",
    "They didn't had lunch yet.", "They haven't had lunch yet.",
    "He is very tired, but he keep working.", "He is very tired, but he keeps working.",
    "This is the house where I was born in.", "This is the house where I was born.",
    "He suggested me that I should join.", "He suggested that I should join.",
    "She do not wants to leave.", "She does not want to leave."
]

# Ground truth labels (0 for Incorrect, 1 for Correct)
expected_labels = [0, 1] * (len(test_cases) // 2)

# Tracking results
correct_predictions = 0
results = []

# Loop through all sentences
for sentence, expected in zip(test_cases, expected_labels):
    classification = classify_sentence(sentence)  # Classify each sentence
    predicted_label = 1 if classification == "Correct" else 0

    # Check if prediction matches expectation
    is_correct = predicted_label == expected
    if is_correct:
        correct_predictions += 1

    # Log the result
    results.append({
        "Sentence": sentence,
        "Expected": "1" if expected == 1 else "0",
        "Predicted": classification,
        "Match": "✅" if is_correct else "❌"
    })

# Calculate accuracy
accuracy = (correct_predictions / len(test_cases)) * 100

# Display Results
print("\n✅ **Sentence Classification Results:**\n")
for i, result in enumerate(results, 1):
    print(f"{i}. Sentence: {result['Sentence']}")
    print(f"   Expected: {result['Expected']}")
    print(f"   Predicted: {result['Predicted']}")
    print(f"   Match: {result['Match']}")
    print("-" * 40)

# Display Overall Accuracy
print(f"\n🎯 **Overall Accuracy: {accuracy:.2f}%**")
print(f"✅ Correct Predictions: {correct_predictions}/{len(test_cases)}")
print(f"❌ Incorrect Predictions: {len(test_cases) - correct_predictions}\n")


✅ **Sentence Classification Results:**

1. Sentence: She will goes to the market yesterday but forgot her wallet.
   Expected: 0
   Predicted: Incorrect
   Match: ✅
----------------------------------------
2. Sentence: She went to the market yesterday, but forgot her wallet.
   Expected: 1
   Predicted: Correct
   Match: ✅
----------------------------------------
3. Sentence: Me and him goes to store and buyed candies.
   Expected: 0
   Predicted: Incorrect
   Match: ✅
----------------------------------------
4. Sentence: He and I went to the store and bought candies.
   Expected: 1
   Predicted: Correct
   Match: ✅
----------------------------------------
5. Sentence: He don't knows nothing about fixing cars.
   Expected: 0
   Predicted: Incorrect
   Match: ✅
----------------------------------------
6. Sentence: He doesn't know anything about fixing cars.
   Expected: 1
   Predicted: Correct
   Match: ✅
----------------------------------------
7. Sentence: Although he was tired. Deci

In [ ]:
model.save_pretrained('/content/drive/MyDrive/ML_project/Classification')
tokenizer.save_pretrained('/content/drive/MyDrive/ML_project/Classification')

('/content/drive/MyDrive/ML_project/Classification/tokenizer_config.json',
 '/content/drive/MyDrive/ML_project/Classification/special_tokens_map.json',
 '/content/drive/MyDrive/ML_project/Classification/vocab.txt',
 '/content/drive/MyDrive/ML_project/Classification/added_tokens.json')